In [ ]:
# # This Python 3 environment comes with many helpful analytics libraries installed
# # It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# # For example, here's several helpful packages to load

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# # Input data files are available in the read-only "../input/" directory
# # For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# # You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# # You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

> # 1. Thêm thư viện và các gói hỗ trợ

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
import cv2
import pandas as pd
import seaborn as sns
import keras
import tensorflow_addons as tfa

from keras.preprocessing.image import ImageDataGenerator
from keras.applications import DenseNet169
from keras.layers import Dense,Dropout,Flatten
from tensorflow.keras.layers import GlobalAveragePooling2D
from keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

> # 2. Đọc các file dữ liệu

In [ ]:
df=pd.read_csv("../input/plant-pathology-2021-fgvc8/train.csv")
df

In [ ]:
df.info()

Ta có 2 tập dữ liệu dùng để huấn luyện:  
1. Tập dữ liệu là file .csv chứa 2 cột:  
- Cột 1 là tên của tất cả các ảnh (**image**)
- Cột 2 là các loại bệnh tương ứng với mỗi chiếc ảnh bên cột 1 (**labels**)  
2. Tập dữ liệu là file ảnh chứa tất cả gồm 18632 ảnh màu và tất cả các ảnh đều có tên trong file .csv ở trên.

> # 3. Tiền xử lý

### Visualize dữ liệu

In [ ]:
ax = plt.subplots(figsize=(18, 6))
sns.set_style("whitegrid")
sns.countplot(x='labels', data=df);
plt.xticks(rotation=90);

In [ ]:
df.labels.value_counts().to_frame().style.background_gradient(cmap="plasma")

In [ ]:
import plotly.graph_objects as go
colors = ['gold', 'mediumturquoise', 'darkorange', 'lightgreen']

label_counts = df['labels'].value_counts()
fig = go.Figure(data=[go.Pie(labels=label_counts.index,values=label_counts)])
fig.update_traces(hoverinfo='label+percent', textinfo='value', textfont_size=20,
                  marker=dict(colors=colors, line=dict(color='#000000', width=2)))
fig.update_layout(title='Labels distribution')
fig.show()
plt.savefig('labels1.png',transparent=True)

### **_NOTE:_**  
Có thể thấy rằng ta có 12 nhãn bệnh trong tập dữ liệu trên. Tuy nhiên, nhiều nhãn bệnh là sự kết hợp của các nhãn bệnh khác với nhau.  
Cho nên, thực tế ta sẽ chỉ có 5 nhãn bệnh:    
* rust
* scab
* complex
* frog_eye_leaf_spot
* powdery_mildew

Và 1 nhãn còn lại là:  
* healthy

Vì **một ảnh (hay 1 lá)** có thể có **nhiều** loại **bệnh (hay nhãn bệnh)** khác nhau cho nên đây là bài toán **Multi-labels Classification!!!**

In [ ]:
df['labels']

Các nhãn đang ở dạng string cho nên chúng em tách string này ra thành list chứa các nhãn bệnh riêng biệt: 

In [ ]:
df['labels']=df['labels'].apply( lambda string: string.split(' ') )
df.head()

Vì những bài toán phân loại ảnh rất quan trọng dữ liệu đầu vào vì với các ảnh chất lượng khác nhau sẽ ảnh hưởng tới kết quả của quá trình huấn luyện mô hình từ đó dẫn đến kết quả phân lớp tốt hay ko tốt.

Hãy cùng xem qua 1 vài ảnh cùng với nhãn và kích thước của ảnh 

In [ ]:
train_path="../input/plant-pathology-2021-fgvc8/train_images"
plt.figure(figsize=(20,40))
i=1
for idx,s in df.head(9).iterrows():
    img_path = os.path.join(train_path,s['image'])
    img=cv2.imread(img_path)
    img=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    
    fig=plt.subplot(9,3,i)
    fig.imshow(img)
    fig.set_title(s['labels'])
    i+=1

> # 4. Đọc và sinh ảnh

Chúng ta có 18632 ảnh cho việc huấn luyện và chúng em chưa biết rằng liệu lượng ảnh này đã là đủ cho mô hình học mà không bị overfitting hay chưa cho nên chúng em đã thử 2 thử nghiệm:
1. 18632 ảnh là đủ:
- Khi thực hiện thử nghiệm này, mô hình về sau của chúng em có khả năng học rất tốt, học gần hoàn hảo 100% từ tập dữ liệu. Tuy nhiên khi chạy predict và nộp kết quả thì score lại vô cùng kém (khoảng 0.2) cho nên chúng em đã kết luận rằng thử nghiệm này là không ổn hay 18632 ảnh là không đủ để training. Và thử nghiệm thứ 2 của chúng em là sẽ Augment thêm ảnh để cho mô hình học.
2. Augment thêm ảnh
- Chúng em sử dụng 1 hàm có sẵn của thư viện keras đó là ImageDataGenerator để tự động augment thêm ảnh. Các tiêu chí sinh thêm ảnh được liệt kê ở block code bên dưới đây:

In [ ]:
datagen = ImageDataGenerator(
    rescale=1/255.0, # scale giá trị của các điểm ảnh về [0, 1.0]
    rotation_range=5, # quay ảnh 1 góc 5 rad
    zoom_range=0.1, # phóng to, thu nhỏ ảnh trong khoảng bằng [0.1, 1.0] so với ảnh gốc
    horizontal_flip=True, # lật ảnh theo chiều ngang
    vertical_flip=True, # lât ảnh theo chiều dọc
    shear_range=0.05, # làm méo ảnh ngẫu nhiên 
    brightness_range=[0.7, 1.3], # tăng giảm độ sáng của ảnh bằng [0.7, 1.3] so với ảnh gốc
    validation_split=0.2 # chia 2 phần train và valid để training cũng như là validating model
)
BATCH_SIZE = 32 # batch size = 32
HEIGHT = 224 # Chiều cao của ảnh
WIDTH = 224 # Chiều rộng của ảnh
CHANNEL = 3 # Số kênh màu của ảnh

Có thể thấy các ảnh có kích thước rất lớn với độ dài, độ rộng khác nhau.  
Để xử lý các ảnh với kích thước lớn như này là vô cùng tốn tài nguyên cũng như là thời gian:
- Thời gian load ảnh lên để xử lý
- Thời gian xử lý 
- Tài nguyên CPU của kaggle là không đủ để xử lý được hết số lượng 18632 ảnh với size > 2500x2500  
Vì vậy, chúng em sử dụng 1 hàm đọc ảnh từ ImageDataGenerator để có thể load ảnh trực tiếp từ thư mục, đó là flow_from_dataframe. Thêm nữa trong hàm này chúng em lấy dữ liệu ảnh là tập ảnh đã được resize về (256,256) để quá trình chạy không tốn kém quá nhiều. 

In [ ]:
train_generator = datagen.flow_from_dataframe(
    df,
    directory='../input/resized-plant2021/img_sz_256',
    subset='training',
    x_col='image',
    y_col='labels',
    target_size=(HEIGHT,WIDTH),
    color_mode='rgb', # 3 kênh màu rgb
    class_mode='categorical',
    batch_size=BATCH_SIZE,
    shuffle=True,
    seed=44
    )

valid_generator = datagen.flow_from_dataframe(
    df,
    directory='../input/resized-plant2021/img_sz_256',
    subset='validation',
    x_col='image',
    y_col='labels',
    target_size=(HEIGHT,WIDTH),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=BATCH_SIZE,
    shuffle=True,
    seed=44
    )

> # 5. Xây dựng mô hình và huấn luyện

Sau khi có dữ liệu ảnh, chúng em sử dụng mô hình mạng DenseNet169 làm mô hình cơ sở để huấn luyện.

In [ ]:
weight_path='../input/keras-pretrain-model-weights/densenet169_weights_tf_dim_ordering_tf_kernels_notop.h5'

Chúng em sử dụng weight của mạng mà không có các tầng fully-connected ở phía trên của mạng DenseNet

Chúng em xây dựng mô hình mạng neuron từ mạng DenseNet169 kết hợp với mạng fully-connected có các tầng với kích thước (256, 128)  
Tầng dense cuối cùng của mô hình sẽ có 6 unit vì đây là bài toán phân lớp với 6 nhãn. Tầng dense 6 unit này sau đấy với 1 threshold nào đó sẽ quyết định xem từng ảnh nào sẽ có 1 hay nhiều nhãn bệnh nào.

In [ ]:
base_model=DenseNet169(weights=weight_path,include_top=False, input_shape=(HEIGHT,WIDTH,CHANNEL))
x=base_model.output
x=GlobalAveragePooling2D()(x)
x=Dense(256,activation='relu')(x)
x=Dropout(0.2)(x)
x=Dense(128,activation='relu')(x)
predictions=Dense(6,activation='sigmoid')(x)

model=Model(inputs=base_model.input,outputs=predictions)

# model.summary()

Tuy nhiên, trước khi train mô hình chúng em sẽ huấn luyện các tầng fully-connected trước.

In [ ]:
for layer in base_model.layers:
    layer.trainable=False

Sử dụng độ đo accuracy và f1 để đánh giá

In [ ]:
f1 = tfa.metrics.F1Score(num_classes=6,average='macro')

model.compile(optimizer=keras.optimizers.Adam(lr=0.001), loss='binary_crossentropy',metrics=[f1])

Sử dụng một vài hàm hỗ trợ việc huấn luyện như EarlyStopping và ReduceLROnPlateau để mô hình có thể học tốt hơn dựa vào việc cải thiện learning rate và khi mô hình không cải thiện được việc học thì mô hình sẽ dừng lại theo cơ chế của EarlyStopping.

In [ ]:
earlyStopping=EarlyStopping(
    patience=5,
    monitor=f1,
    mode='max',
    restore_best_weights=True
)
lrSchedule = ReduceLROnPlateau(
    monitor='val_f1_score', 
    factor=0.05, 
    patience=4, 
    verbose=1
)

Huấn luyện mô hình

In [ ]:
hist = model.fit_generator(
    generator=train_generator,
    validation_data=valid_generator,
    epochs=30,
    steps_per_epoch=train_generator.samples//128,
    validation_steps=valid_generator.samples//128,
    callbacks=[earlyStopping, lrSchedule]
)

In [ ]:
# for i, layer in enumerate(model.layers):
#     print(i, layer.name, "-", layer.trainable)

In [ ]:
model.layers[595:]

freeze các tầng đã train từ trước lại và tiếp tục train các tầng còn lại

In [ ]:
for layer in model.layers:
    layer.trainable=True

model.compile(optimizer=keras.optimizers.Adam(lr=0.001), loss='binary_crossentropy',metrics=[f1])
history = model.fit_generator(generator=train_generator,
                    validation_data=valid_generator,
                    epochs=30,
                    steps_per_epoch=train_generator.samples//128,
                    validation_steps=valid_generator.samples//128,
                    callbacks=[earlyStopping, lrSchedule])

Biểu thị lại điểm accuracy theo từng epoch

In [ ]:
# plt.figure(figsize=(15,6))
# epoch_list = list(range(1, len(history.history['accuracy']) + 1))
# plt.plot(epoch_list, history.history['accuracy'],label='accuracy')
# plt.plot(epoch_list, history.history['val_accuracy'],label='val_accuracy')
# plt.xlabel('epoches')
# plt.ylabel('accuracy')
# plt.legend()
# plt.show()

Biểu thị lại điểm f1 theo từng epoch

In [ ]:
plt.figure(figsize=(15,6))
epoch__list = list(range(1,len(history.history['f1_score'])+1))
plt.plot(epoch__list, history.history['f1_score'],label='f1_score')
plt.plot(epoch__list, history.history['val_f1_score'],label='val_f1_score')
plt.xlabel('epoches')
plt.ylabel('f1')
plt.legend()
plt.show()

In [ ]:
model.save('plant_densenet169_ver02.h5')

> # 6. Dự đoán trên tập test_images và nộp bài

Đọc dữ liệu file sample_submission

In [ ]:
sample_sub = pd.read_csv('../input/plant-pathology-2021-fgvc8/sample_submission.csv')
sample_sub

Lấy ảnh từ file test_images để dự đoán

In [ ]:
test_data = datagen.flow_from_dataframe(
    sample_sub,
    directory='../input/plant-pathology-2021-fgvc8/test_images',
    x_col='image',
    y_col=None,
    color_mode='rgb',
    target_size=(HEIGHT,WIDTH),
    class_mode=None,
    shuffle=False
)

predictions = model.predict(test_data)
print(predictions)

class_idx=[]
for pred in predictions:
    pred=list(pred)
    temp=[]
    for i in pred:
        if (i>0.3):
            temp.append(pred.index(i))
    if (temp!=[]):
        class_idx.append(temp)
    else:
        temp.append(np.argmax(pred))
        class_idx.append(temp)
print(class_idx)

In ra kết quả dự đoán

In [ ]:
class_dict = train_generator.class_indices
def get_key(val):
    for key,value in class_dict.items():
        if (val==value):
            return key
print(class_dict)

sub_pred=[]
for img_ in class_idx:
    img_pred=[]
    for i in img_:
        img_pred.append(get_key(i))
    sub_pred.append( ' '.join(img_pred))
print(sub_pred)

In [ ]:
sub = sample_sub[['image']]
sub['labels']=sub_pred
sub

In ra file submission.csv để nộp bài

In [ ]:
sub.to_csv('submission.csv',index=False)